# Xception

# Config

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Imports

In [2]:
from datetime import datetime
import pathlib
import typing as t

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.applications import xception
from tensorflow.keras import preprocessing
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

# Constants

In [3]:
DATASET_PATH = pathlib.Path("../../../datasets/celeb_df/")
MODELS_PATH = pathlib.Path("../../../saved_models")
# TensorBoard logs path
LOGS_PATH = pathlib.Path("./logs")
IMAGE_SIZE = (256, 256)
INPUT_SHAPE = (*IMAGE_SIZE, 3)

# Load datasets

In [4]:
SEED = int(datetime.today().timestamp())
VALIDATION_SPLIT = 0.05
BATCH_SIZE = 32

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
)

Found 2342158 files belonging to 2 classes.
Using 2225051 files for training.


In [6]:
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
)

Found 2342158 files belonging to 2 classes.
Using 117107 files for validation.


In [7]:
REALS_TO_FAKE_RATIO = (
    len(list(DATASET_PATH.joinpath("reals").iterdir()))
    / len(list(DATASET_PATH.joinpath("fakes").iterdir()))
)
REALS_TO_FAKE_RATIO

0.10647836701990959

# Define model

### Load pretrained model

In [8]:
backbone = applications.Xception(
    include_top=False,
    weights="imagenet",
)

### Add preprocessing layer

In [9]:
# Preprocessing
x = layers.Lambda(xception.preprocess_input)(backbone.input)
x = backbone(x)

# Model top
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = tf.keras.layers.Dense(1, activation="sigmoid")(x)

In [10]:
# this is the model we will train
model = tf.keras.Model(inputs=backbone.input, outputs=predictions)

### Freeze pretrained layers

In [11]:
for layer in backbone.layers:
    layer.trainable = False

# Training

In [13]:
METRICS = [
        metrics.BinaryAccuracy(),
        metrics.AUC(),
        #metrics.Precision(),
        #metrics.Recall(),
        #metrics.TruePositives(),
        #metrics.TrueNegatives(),
        #metrics.FalsePositives(),
        #metrics.FalseNegatives(),
    ]

In [15]:
log_dir = LOGS_PATH.joinpath("fit").joinpath(datetime.now().strftime("%Y%m%d-%H%M%S"))

CALLBACKS = [
    callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
    ),
    callbacks.ModelCheckpoint(
        filepath=MODELS_PATH.joinpath("xception_{epoch:02d}_{val_loss:.2f}.h5"),
        monitor='val_loss',
        save_best_only=True,
    ),
    callbacks.TensorBoard(log_dir=log_dir),
]

In [16]:
# Params following "FaceForensics++: Learning to Detect Manipulated Facial Images"
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0002,
    epsilon=1e-08
)

In [19]:
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=METRICS
)

### Initial training

In [20]:
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    class_weight={
        # 0 are fakes since they are first in alphabetical ordering
        0: REALS_TO_FAKE_RATIO,
        1: 1.0,
    },
    epochs=1,
    callbacks=CALLBACKS
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


c:\users\ciche\desktop\projects\dfd\venv\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


  831/69533 [..............................] - ETA: 2:20:39 - loss: 0.1344 - binary_accuracy: 0.5314 - auc: 0.5422

KeyboardInterrupt: 